In [1]:
from sklearn import datasets
import scanpy as sc
import optax
import matplotlib.pyplot as plt
from sklearn import preprocessing as pp
import numpy as np

import ott
import sklearn
import matplotlib.pyplot as plt
from ott.geometry import geometry, pointcloud
import jax
from typing import Mapping, Any, Optional, Union, Callable, Tuple
from types import MappingProxyType
import jax.numpy as jnp
from functools import partial
from ott.solvers.linear import sinkhorn
from ott.problems.linear import linear_problem
from entot.models.model import OTFlowMatching
from entot.nets.nets import MLP_vector_field, MLP_bridge, MLP_marginal,MLP_fused_vector_field
import sklearn.preprocessing as pp
import scanpy as sc
from ott.solvers.linear import sinkhorn, acceleration
from sklearn import preprocessing as pp

/home/icb/dominik.klein/mambaforge/envs/entot_pip/lib/python3.11/site-packages/equinox/_ad.py:753: UserWarning: As of Equinox 0.10.7, `equinox.filter_custom_vjp.defvjp` is deprecated in favour of `.def_fwd` and `.def_bwd`. This new API supports symbolic zeros, which allow for more efficient autodifferentiation rules. In particular:
- the fwd and bwd functions take an extra `perturbed` argument, which     indicates which primals actually need a gradient. You can use this     to skip computing the gradient for any unperturbed value. (You can     also safely just ignore this if you wish.)
- `None` was previously passed to indicate a symbolic zero gradient for     all objects that weren't inexact arrays, but all inexact arrays     always had an array-valued gradient. Now, `None` may also be passed     to indicate that an inexact array has a symbolic zero gradient.
  warnings.warn(
2023-09-12 09:26:19.998327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not 

In [2]:
adata = sc.read("../../data/adata_pancreas_2019.h5ad")

In [3]:
adata = adata[adata.obs["celltype"]!="Multipotent"].copy()
adata.obs["lineage"] = adata.obs.apply(lambda x: "A" if x["celltype"] in ["Acinar", "Tip"] else "ED", axis=1).astype("category")

In [5]:
adata[adata.obs["day"]=="14.5"].obs.groupby(["lineage"]).count()

,day,n_counts,log_counts,n_genes,mt_frac,proliferation,G2M_score,S_score,clusters_fig3_final,clusters_fig3_final_noep,clusters_fig4_final,clusters_fig2_final,clusters_fig6_broad_final,clusters_fig6_fine_final,clusters_fig6_alpha_final,celltype
lineage,,,,,,,,,,,,,,,,
A,3416,3416,3416,3416,3416,3416,3416,3416,3416,3416,3416,3416,3416,3416,155,3416
ED,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203,6203


In [6]:
adata[adata.obs["day"]=="15.5"].obs.groupby(["lineage"]).count()

,day,n_counts,log_counts,n_genes,mt_frac,proliferation,G2M_score,S_score,clusters_fig3_final,clusters_fig3_final_noep,clusters_fig4_final,clusters_fig2_final,clusters_fig6_broad_final,clusters_fig6_fine_final,clusters_fig6_alpha_final,celltype
lineage,,,,,,,,,,,,,,,,
A,7136,7136,7136,7136,7136,7136,7136,7136,7136,7136,7136,7136,7136,7136,6,7136
ED,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749


In [7]:
7136/3749, 3416/6203

(1.9034409175780207, 0.5507012735773013)